In [1]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import numpy as np

from selenium import webdriver
from selenium.webdriver.chrome.service import Service


def get_driver(use_proxy=False):

    chrome_options = Options()
    chrome_options = webdriver.ChromeOptions()

    if use_proxy:
        driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
        driver.get("https://sslproxies.org/")
        driver.execute_script("return arguments[0].scrollIntoView(true);", WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//table[@class='table table-striped table-bordered dataTable']//th[contains(., 'IP Address')]"))))
        ips = [my_elem.get_attribute("innerHTML") for my_elem in WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@class='table table-striped table-bordered dataTable']//tbody//tr[@role='row']/td[position() = 1]")))]
        ports = [my_elem.get_attribute("innerHTML") for my_elem in WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@class='table table-striped table-bordered dataTable']//tbody//tr[@role='row']/td[position() = 2]")))]
        proxies = []
        for i in range(0, len(ips)):
            proxies.append(ips[i]+':'+ports[i])
        chrome_options = webdriver.ChromeOptions()
        i = int(np.random.randint(low=0, high=5, size=(1,)))
        chrome_options.add_argument('--proxy-server={}'.format(proxies[i]))
        chrome_options.add_argument("start-maximized")
        chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
        chrome_options.add_experimental_option('useAutomationExtension', False)

    chrome_options.add_argument("start-maximized")
    service = Service()
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source":
        "const newProto = navigator.__proto__;"
        "delete newProto.webdriver;"
        "navigator.__proto__ = newProto;"
    })

    return driver

In [2]:
driver = get_driver()

Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)


In [6]:
BASE_URL = "https://www.levels.fyi/t"

role = "data-scientist"
location = "singapore"

composed_url = f"{BASE_URL}/{role}/locations/{location}?limit=50"
composed_url

'https://www.levels.fyi/t/data-scientist/locations/singapore?limit=50'

In [8]:
driver.get(composed_url)

In [9]:
html = driver.page_source
soup = BeautifulSoup(html)
dict_data = dict()

In [29]:
soup.find_all("div", class_="MuiTableContainer-root")[0].text#.split("|")#.split("\xa0")[0]#.split(" | ")

'CompanyLocation | DateLevel NameTagYears of ExperienceTotal / At CompanyTotal Compensation (SGD)Base | Stock (yr) | Bonus Illumina Singapore, SG, Singapore | a day agoP3ML / AI5  yrs 3  yrs138.000\xa0S$130\xa0mil | 8\xa0mil | N/ATencent Singapore, SG, Singapore | 3 days agoT11Data5  yrs 2  yrs290.001\xa0S$200\xa0mil | 40\xa0mil | 50\xa0milGrab Singapore, SG, Singapore | 4 days agoG4Analytics7  yrs 4  yrs172.001\xa0S$133\xa0mil | 20\xa0mil | 19\xa0milInfoCepts Singapore, SG, Singapore | 6 days agoL2Data3  yrs 3  yrs57.500\xa0S$57,5\xa0mil | N/A | N/AGrab Singapore, SG, Singapore | 7 days agoG5hidden7  yrs 3  yrs305.001\xa0S$180\xa0mil | 89\xa0mil | 36\xa0milReview your resume with a recruiterStart ReviewStartGrab Singapore, SG, Singapore | 01/08/2024G6Data Science8  yrs 5  yrs332.001\xa0S$192\xa0mil | 100\xa0mil | 40\xa0milNanyang Technolog... Singapore, SG, Singapore | 01/08/2024Project ManagerAI/Computer Vision6  yrs 6  yrs93.600\xa0S$93,6\xa0mil | N/A | N/AHPE Singapore, SG, Singapo

In [32]:
list_scraped = []

for i in range(100, 400, 50):
    print(f"Scrapping offset {i}")
    # Abrimos el navegador en esta pagina
    driver.get(composed_url + f"&offset={i}")
    # Te 'descarga' la pagina como un string
    html = driver.page_source
    # Te convierte el string con HTML dentro en un objeto de BeautifulSoup con metodos handys
    soup = BeautifulSoup(html)
    # Buscamos la tabla de la pagina
    tabla_iteracion = soup.find_all("div", class_="MuiTableContainer-root")[0].text
    # LO añadimos a nuestra lista
    list_scraped.append(tabla_iteracion)
    # Ponemos un pequeño sleep para que la pagina no detecte que es un bot
    import time 
    time.sleep(1)
    

Scrapping offset 100
Scrapping offset 150
Scrapping offset 200
Scrapping offset 250
Scrapping offset 300
Scrapping offset 350


In [42]:
list_scraped[5]

'CompanyLocation | DateLevel NameTagYears of ExperienceTotal / At CompanyTotal Compensation (SGD)Base | Stock (yr) | Bonus Grab Singapore, SG, Singapore | 16/07/2021G5Data6  yrs 1  yr135.428\xa0S$108,3\xa0mil | 13,5\xa0mil | 13,5\xa0milGrab Singapore, SG, Singapore | 30/06/2021G4Product3  yrs 3  yrs126.200\xa0S$94,2\xa0mil | 10\xa0mil | 22\xa0milShopee Singapore, SG, Singapore | 27/06/2021L3General1  yr 1  yr126.209\xa0S$91,3\xa0mil | 18,8\xa0mil | 16,1\xa0milShopee Singapore, SG, Singapore | 13/06/2021SeniorGeneral4  yrs 2  yrs160.413\xa0S$82,2\xa0mil | 67,6\xa0mil | 10,6\xa0milGojek Tech Singapore, SG, Singapore | 31/05/2021L3-2  yrs 2  yrs96.000\xa0S$96\xa0mil | N/A | N/ANegotiate up, or your money back+$25k+$19k+$13kNegotiate your offerMake MoreIndeed Singapore, SG, Singapore | 31/05/2021L1-3  yrs 0  yrs140.250\xa0S$115\xa0mil | 13,3\xa0mil | 12\xa0milFacebook Singapore, SG, Singapore | 30/05/2021IC4Legal3  yrs 0  yrs133.652\xa0S$113,8\xa0mil | 7,9\xa0mil | 11,9\xa0milShopee Singap